# Saliency using Gradient Backpropagation

Implement the saliency algorithm described in https://arxiv.org/pdf/1312.6034.pdf.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

! cp -r '/content/drive/MyDrive/Colab Notebooks/cs-no/6.grads/config' .

Mounted at /content/drive
     |████████████████████████████████| 706kB 9.0MB/s 


In [22]:
import tensorflow as tf
from config import config, data, model, utils

from config.utils import to_image, normalize

utils.setup_clean_image_plotting()

In [5]:
images = data.load_images()
x = model.preprocess(images)

In [6]:
nn = model.build()

logits = nn(x, training=False)
preds = tf.argmax(logits, axis=1)
probs = tf.nn.softmax(logits)
decoded = model.decode(probs.numpy(), top=2)

40960/35363 [==================================] - 0s 0us/step


In [7]:
utils.plot(utils.to_image(images),
           titles=[f'{f}\n{p[0][1]} {p[0][2]:.0%}'
                  for f, p in zip(data.class_names, decoded)],
           rows=4)

Output hidden; open in https://colab.research.google.com to view.

In [8]:
def activation_loss(y, units):
    return tf.gather(y, units, axis=1, batch_dims=1)

@tf.function
def gradients(inputs, units):
    with tf.GradientTape() as tape:
        tape.watch(inputs)
        y = nn(inputs)
        loss = activation_loss(y, units)
    
    grads = tape.gradient(loss, inputs)

    return loss, grads

In [23]:
_, g = gradients(x, preds)

sa = normalize(tf.reduce_sum(tf.abs(g), axis=-1))
sp = normalize(tf.reduce_sum(tf.nn.relu(g), axis=-1))
sn = normalize(tf.reduce_sum(tf.nn.relu(-g), axis=-1))

In [30]:
columns = [
    to_image(images),
    sa.numpy(),
    sp.numpy(),
    sn.numpy(),
    to_image(images * sa[..., tf.newaxis])
]
titles = ['original',
          'absolute gradients',
          'positive gradients',
          'negative gradients',
          'original * abs(gradients)']

utils.plot(sum(zip(*columns), ()),
           titles=titles,
           rows=len(images))

Output hidden; open in https://colab.research.google.com to view.